In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from google.colab import drive
from sklearn.model_selection import train_test_split

In [ ]:
TEAM_MEMBER_ID = 3

print(f"--- SETUP FOR TEAM MEMBER {TEAM_MEMBER_ID} ---")
print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

--- SETUP FOR TEAM MEMBER 3 ---
TensorFlow Version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
print("\nMounting Google Drive...")
drive.mount('/content/drive')


Mounting Google Drive...
Mounted at /content/drive


In [ ]:
DRIVE_PATH = "/content/drive/My Drive/"
ZIP_FILE_PATH = os.path.join(DRIVE_PATH, "final_data.zip")
PARTITION_FILE = os.path.join(DRIVE_PATH, f"partition_processed_{TEAM_MEMBER_ID}.json")
MODEL_SAVE_PATH = os.path.join(DRIVE_PATH, f"keras_cnn_model_{TEAM_MEMBER_ID}.keras")

In [ ]:
print(f"\nCopying dataset from {ZIP_FILE_PATH}...")
!cp "{ZIP_FILE_PATH}" /content/
print("Unzipping local dataset...")
!unzip -o /content/final_data.zip -d /content/ > /dev/null
print("Dataset ready on local disk.")


Copying dataset from /content/drive/My Drive/final_data.zip...
Unzipping local dataset...
Dataset ready on local disk.


In [ ]:
print(f"\nLoading your unique data partition from: {PARTITION_FILE}")
with open(PARTITION_FILE, 'r') as f:
    partition_data = json.load(f)
my_files = partition_data['files']
my_labels = partition_data['labels']
print(f"Successfully loaded {len(my_files)} unique image paths.")


Loading your unique data partition from: /content/drive/My Drive/partition_processed_3.json
Successfully loaded 30000 unique image paths.


In [ ]:
print("\n--- Building `tf.data` Pipeline ---")

train_files, val_files, train_labels, val_labels = train_test_split(
    my_files, my_labels, test_size=0.2, random_state=42, stratify=my_labels
)

IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def parse_image(filename, label):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = image / 255.0
    return image, label

def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

def create_dataset(files, labels, is_training=True):
    dataset = tf.data.Dataset.from_tensor_slices((files, labels))
    dataset = dataset.map(parse_image, num_parallel_calls=AUTOTUNE)
    if is_training:
        dataset = dataset.shuffle(buffer_size=1000)
        dataset = dataset.map(augment_image, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    return dataset

train_ds = create_dataset(train_files, train_labels)
val_ds = create_dataset(val_files, val_labels, is_training=False)
print("Training and validation datasets are ready!")


--- Building `tf.data` Pipeline ---
Training and validation datasets are ready!


In [ ]:
print("\n--- Defining Model Architecture and Loading Checkpoint if Available ---")

def build_model(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1)
    ])
    return model


--- Defining Model Architecture and Loading Checkpoint if Available ---


In [ ]:
if os.path.exists(MODEL_SAVE_PATH):
    print(f"Checkpoint found! Loading full model from: {MODEL_SAVE_PATH}")
    model = tf.keras.models.load_model(MODEL_SAVE_PATH)
else:
    print("No checkpoint found. Building a new model from scratch.")
    model = build_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='mae',
        metrics=['mae']
    )


No checkpoint found. Building a new model from scratch.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
print("\n--- Setting Up Training ---")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_SAVE_PATH, monitor='val_mae', mode='min',
    save_best_only=True, save_weights_only=False, verbose=1
)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_mae', mode='min', patience=5, verbose=1, restore_best_weights=True
)
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_mae', mode='min', factor=0.2, patience=2, verbose=1
)

NUM_EPOCHS = 40
print(f"\nStarting training for Model {TEAM_MEMBER_ID} for up to {NUM_EPOCHS} epochs...")

history = model.fit(
    train_ds,
    epochs=NUM_EPOCHS,
    validation_data=val_ds,
    callbacks=[checkpoint_callback, early_stopping_callback, reduce_lr_callback]
)

print(f"\nTraining complete! Your best model has been saved to: {MODEL_SAVE_PATH}")


--- Setting Up Training ---

Starting training for Model 3 for up to 40 epochs...
Epoch 1/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 40.0487 - mae: 40.0487
Epoch 1: val_mae improved from inf to 19.27037, saving model to /content/drive/My Drive/keras_cnn_model_3.keras
750/750 ━━━━━━━━━━━━━━━━━━━━ 71s 76ms/step - loss: 40.0358 - mae: 40.0358 - val_loss: 19.2704 - val_mae: 19.2704 - learning_rate: 0.0010
Epoch 2/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 19.8805 - mae: 19.8805
Epoch 2: val_mae did not improve from 19.27037
750/750 ━━━━━━━━━━━━━━━━━━━━ 55s 72ms/step - loss: 19.8797 - mae: 19.8797 - val_loss: 19.2913 - val_mae: 19.2913 - learning_rate: 0.0010
Epoch 3/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 18.2787 - mae: 18.2787
Epoch 3: val_mae improved from 19.27037 to 17.28441, saving model to /content/drive/My Drive/keras_cnn_model_3.keras
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 74ms/step - loss: 18.2784 - mae: 18.2784 - val_loss: 17.2844 - val_mae: 17.2844 - 